In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df=pd.read_csv('/content/dataset/IMDB.csv')
df.head()

In [ ]:
df['review'][0]

In [ ]:
# pre-processing the dataset
#df=df.sample(10000)
#df.shape
#df.info()

df['sentiment'].replace({'positive':1, 'negative':0},inplace=True)

# Cleaning the dataset
import re
clean=re.compile('<.*?>')
re.sub(clean,'',df.iloc[2].review) # remove symbols

def clean_html(text):
    clean=re.compile('<.*?>')
    return re.sub(clean, '', text)
df['review']=df['review'].apply(clean_html) # remove html tag


import nltk
from nltk.corpus import stopwords
import string
from nltk.stem.porter import PorterStemmer
porter=PorterStemmer()

# remove stop words

def transform_review(review):
    review = review.lower()
    review = nltk.word_tokenize(review)
    
    y = []
    for i in review:
        if i.isalnum():
            y.append(i) 
    review = y[:]
    y.clear()
    
    for i in review:
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(i)           
    review = y[:]
    y.clear()

    for i in review:
        y.append(ps.stem(i))
              
    return " ".join(y)

df['review'] = df['review'].apply(transform_review)

In [ ]:
# tokenization
# Using wordcloud

from wordcloud import WordCloud
wc = WordCloud(width=500,height=500,min_font_size=10,background_color='white')

positive_corpus = []
for msg in df[df['sentiment'] == 1]['review'].tolist():
    for word in msg.split():
        positive_corpus.append(word)       


In [ ]:
from collections import Counter
sns.barplot(pd.DataFrame(Counter(positive_corpus).most_common(30))[0],pd.DataFrame(Counter(positive_corpus).most_common(30))[1])
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
# vectorizing
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=4500)
X=tfidf.fit_transform(df['review']).toarray()

y=df.iloc[:,-1].values

In [ ]:
# Split dataset for training and testing
from  sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=0.2) # this size can be changed for different portions of the training data
X_train.shape
X_test.shape
y_train.shape
y_test.shape

In [ ]:
# navie bay model
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score
cl1=GaussianNB()
cl2=MultinomialNB()
cl3=BernoulliNB()
cl1.fit(X_train,y_train)
cl2.fit(X_train,y_train)
cl3.fit(X_train,y_train)

ypre1=clf1.predict(X_test)
ypre2=clf2.predict(X_test)
ypre3=clf3.predict(X_test)

from sklearn.metrics import accuracy_score
print("Gausssian",accuracy_score(y_test,y_pred1))
print("Multinomial",accuracy_score(y_test,y_pred2))
print("Bernaulli",accuracy_score(y_test,y_pred3))

# save results
import pickle
pickle.dump(tfidf,open('vectorizer1.pkl','wb'))
pickle.dump(clf2,open('model.pkl','wb'))